In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np
sc = pyspark.SparkContext('local[*]')

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)
test = pd.read_csv("Sources/test_final_100k.csv")

In [3]:
mas_features = pd.read_csv("Sources/mas_features.csv")
postulaciones = postulaciones.merge(mas_features, on='idaviso')
test = pd.concat([test,postulaciones])
test = test.drop_duplicates(['idaviso','idpostulante'])
postulaciones = test[100000:].drop('id', axis=1)
test = test[:100000].drop('id', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

postulaciones.head()

,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,idaviso,idpostulante,prom_edad,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,125.0,94.0,0.0,FEM,4631.0,1112248724,NjlD,30.251142,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,206236,37.0,FEM,12
1,125.0,94.0,0.0,FEM,4631.0,1112248724,MV6eM8N,30.251142,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,189555,27.0,MASC,11
2,1773.0,977.0,24.0,FEM,8583.0,1112304156,MV6eM8N,29.206545,1.0,Full-time,Junior,Comunicacion,RH Talentum,34,189555,27.0,MASC,11
3,381.0,319.0,6.0,NaN,18392.0,1112445502,MV6eM8N,30.327611,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,34,189555,27.0,MASC,11
4,361.0,267.0,3.0,FEM,5080.0,1112254843,MV6eM8N,29.357257,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,64,189555,27.0,MASC,11


In [5]:

postulaciones.head()

,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,idaviso,idpostulante,prom_edad,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,125.0,94.0,0.0,FEM,4631.0,1112248724,NjlD,30.251142,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,206236,37.0,FEM,12
1,125.0,94.0,0.0,FEM,4631.0,1112248724,MV6eM8N,30.251142,1.0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,33,189555,27.0,MASC,11
2,1773.0,977.0,24.0,FEM,8583.0,1112304156,MV6eM8N,29.206545,1.0,Full-time,Junior,Comunicacion,RH Talentum,34,189555,27.0,MASC,11
3,381.0,319.0,6.0,NaN,18392.0,1112445502,MV6eM8N,30.327611,1.0,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,34,189555,27.0,MASC,11
4,361.0,267.0,3.0,FEM,5080.0,1112254843,MV6eM8N,29.357257,1.0,Full-time,Senior / Semi-Senior,E-commerce,ASSIST CARD,64,189555,27.0,MASC,11


In [6]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,idaviso,idpostulante,prom_edad,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,NaN,NaN,NaN,NaN,NaN,739260,6M9ZQR,NaN,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,69266,42.0,FEM,21
1,NaN,NaN,NaN,NaN,NaN,739260,6v1xdL,NaN,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,83910,31.0,MASC,9
2,NaN,NaN,NaN,NaN,NaN,739260,ezRKm9,NaN,1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,351736,36.0,FEM,12
3,NaN,NaN,NaN,NaN,NaN,758580,1Q35ej,NaN,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,15447,69.0,MASC,15
4,NaN,NaN,NaN,NaN,NaN,758580,EAN4J6,NaN,1,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,138590,32.0,FEM,11


def dates_format(date):
    try:
        date_new = pd.to_datetime('today').year-pd.to_datetime(date).year
        #date_new = int(date_new)
        return date_new
    except ValueError:
        return -1

fechasRDD = sc.parallelize(postulaciones['fechanacimiento'])
postulaciones['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = sc.parallelize(test['fechanacimiento'])
test['fechanacimiento'] = fechasRDD.map(lambda x: dates_format(x)).collect()
fechasRDD = None

In [7]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.head()

,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,prom_edad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,NaN,NaN,NaN,NaN,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,69266,42.0,FEM,21
1,NaN,NaN,NaN,NaN,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,83910,31.0,MASC,9
2,NaN,NaN,NaN,NaN,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,7,351736,36.0,FEM,12
3,NaN,NaN,NaN,NaN,NaN,NaN,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,15447,69.0,MASC,15
4,NaN,NaN,NaN,NaN,NaN,NaN,Full-time,Otro,Tecnologia / Sistemas,TeleTech,16,138590,32.0,FEM,11


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in postulaciones.columns:
    if not postulaciones[col].dtype == 'O': continue
    print(col)
    postulaciones[col] = postulaciones[col].fillna('na')
    test[col] = test[col].fillna('na')
    le.fit(pd.concat([postulaciones[col],test[col]]))
    test[col] = le.transform(test[col])
    postulaciones[col] = le.transform(postulaciones[col])
postulaciones.head()

TendenciaSexo
tipo_de_trabajo
nivel_laboral
nombre_area
denominacion_empresa
sexo


,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,prom_edad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
0,125.0,94.0,0.0,0,4631.0,30.251142,1,4,120,2407,33,206236,37.0,1,12
1,125.0,94.0,0.0,0,4631.0,30.251142,1,4,120,2407,33,189555,27.0,2,11
2,1773.0,977.0,24.0,0,8583.0,29.206545,1,2,36,3211,34,189555,27.0,2,11
3,381.0,319.0,6.0,2,18392.0,30.327611,1,4,36,2208,34,189555,27.0,2,11
4,361.0,267.0,3.0,0,5080.0,29.357257,1,4,64,153,64,189555,27.0,2,11


In [9]:
postulaciones.sample(5)
from sklearn.utils import shuffle
postulaciones = shuffle(postulaciones)

In [10]:
test.sample(5)

,FEM,MASC,NO_DECLARA,TendenciaSexo,Unnamed: 0,prom_edad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,Unnamed: 0.1,fechanacimiento,sexo,nivel
90467,NaN,NaN,NaN,2,NaN,NaN,1,4,78,1595,4,180546,26.0,2,11
70737,NaN,NaN,NaN,2,NaN,NaN,1,1,39,3123,69,318843,31.0,2,12
48768,NaN,NaN,NaN,2,NaN,NaN,1,4,185,2629,67,173697,21.0,2,11
2452,NaN,NaN,NaN,2,NaN,NaN,2,4,185,3158,26,435076,29.0,2,11
14285,NaN,NaN,NaN,2,NaN,NaN,1,4,91,3158,22,269977,64.0,2,15


In [11]:
postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [12]:
aviso = None
postulantes = None
postulaciones.fillna(-1,inplace=True)
test.fillna(-1,inplace=True)
test.drop([5],axis=1,inplace=True)
postulaciones.drop([5],axis=1,inplace=True)

In [13]:
test.drop([12],axis=1,inplace=True)
postulaciones.drop([12],axis=1,inplace=True)

In [14]:
postulaciones.head()

,1,2,3,4,6,7,8,9,10,11,13,14,15
2873756,13.0,5.0,0.0,0,30.666667,5,3,119,1413,2,27.0,1,12
8894472,704.0,216.0,13.0,0,26.883822,2,2,174,1204,18,23.0,1,3
6009406,13.0,34.0,0.0,1,32.234043,1,4,30,2538,7,31.0,1,10
8914920,18.0,882.0,6.0,1,34.381798,1,4,5,891,46,48.0,2,6
7750914,122.0,260.0,1.0,1,36.973753,1,4,30,1159,30,25.0,2,3


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#X_train, X_test, y_train, y_test = train_test_split(postulaciones, resultado, test_size=0.2)
clf=RandomForestClassifier(n_estimators=50,n_jobs=-1,warm_start=True,criterion='entropy')
clf.fit(postulaciones,resultado)
y_pred=clf.predict(test)



In [16]:
test = pd.read_csv('Sources/test_final_100k.csv')
test['sepostulo'] = y_pred
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.to_csv('Sources/resultados_random.csv', index=False)

In [17]:
y_pred[y_pred==1].shape

(76696,)

In [18]:
test.head()

,id,sepostulo
0,0,1.0
1,1,1.0
2,2,0.0
3,3,1.0
4,4,1.0
